In [30]:
%load_ext autoreload
%autoreload 2

In [33]:
from dataclasses import dataclass, field

from typing import Dict, List, Tuple, Optional, Callable, Any

import numpy as np
from PIL import Image

from pysolotools.consumers import Solo
from pysolotools.converters.solo2coco import SOLO2COCOConverter
from pysolotools.core.models import KeypointAnnotationDefinition, RGBCameraCapture
from pysolotools.core.models import BoundingBox2DLabel, BoundingBox2DAnnotation
from pysolotools.core.models import BoundingBox3DLabel, BoundingBox3DAnnotation
from pysolotools.core.models import Frame, Capture
from scipy.spatial.transform import Rotation as R

from torchvision.datasets import ImageFolder
from torchvision.models import swin_v2_t, Swin_V2_T_Weights
# from torch.utils.data import ConcatDataset, DataLoader
from collections import OrderedDict
from torch.utils.data import DataLoader
from scipy.io import savemat, loadmat
import torch.nn.functional as F

import torch
from torch import nn, Tensor
from torchvision.ops import FeaturePyramidNetwork

import lightning.pytorch as pl

import matplotlib.pyplot as plt

import transformer
import reader
import network
# from my_trainer import LitCVMPCA, HungarianMatcher, SetCriterion

# %matplotlib ipympl

In [37]:
train_folder = 'D:/Unity/dataset/solo'
train_loader = reader.UnityDataset.from_unity_to_loader(train_folder, batch_size=4)

Dataset loaded succesfully.


In [38]:
backbone = transformer.Swin(is_trainable=False)
backbone.cuda()
backbone.eval()

with torch.no_grad():
    for i, sample in enumerate(train_loader.dataset):
        path_dict, image_dict, _ = sample
        image_dict = {key: item[None, ...].cuda() for key, item in image_dict.items()}
        for key, image in image_dict.items():  # all images in 
            path = path_dict[key]
            _path = f'{path[:path.rfind("/")]}/{key}.mat'
            outputs = backbone(image)
            output_dict = {str(i): output[0].cpu().numpy() for i, output in enumerate(outputs)}
            savemat(_path, output_dict)

        print(f'{i/len(train_loader.dataset)*100:.2f}%', end='\r')

        break


In [64]:
train_loader = reader.UnityMatDataset.from_unity_to_loader(train_folder, batch_size=4)

for batch in train_loader:
    images, targets = batch
    
    break

Loading: 99.50%

In [52]:
for sample in train_loader.dataset:
    path_dict, images, targets = sample
    
    break

Loading: 99.50%

In [32]:
B, H, W, C = 4, 8, 8, 16
inp = torch.rand((B, C, H, W))
pixel_coor = torch.rand((4, 1, 2))

sampled_inp = F.grid_sample(inp, pixel_coor[None, ...], align_corners=True)
sampled_inp.shape

torch.Size([4, 100, 16])